# Backpropagation through Back substitution with a Backslash
Notebook per il seminario d'esame per il corso di Metodi di Approssimazione 

## Algebra degli operatori

Definiamo il tipo di dato custom che rappresenta la funzione lineare

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

𝒪 (generic function with 1 method)

Dobbiamo anche fare *overloading* delle operazioni

In [2]:
import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator

adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper

-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪()), X::Matrix) = -X  # 𝒪() - X should be -X
\(ℐ::typeof(ℐ()), A::Matrix) = A    # left division with ℐ() does nothing
*(A::Operator, X::Matrix) = A.op(X)
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

## Una semplice rete neurale

In [3]:
using OffsetArrays

# funzione di attivazione
h(x)  = tanh(x)
h′(x) = 1 - h(x)^2


# questa funzione, di fatto, effettua il forward pass
function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

Adesso possiamo costruire la rete

In [4]:
## Un po' di parametri che descrivono la rete...

n = [5,4,3,1]   # contiene [n₀...n_N], le dimensioni dei layer
k = 10          # batchsize
N = length(n)-1 # numero di layer (nascosti + output). Dev'essere positivo

init(sizes...) = 0.01randn(sizes...)    # utility function per inizializzare

init (generic function with 1 method)

In [5]:
## Un po' di parametri degni di questo nome

# Creiamo le matrici dei pesi e i vettori dei bias
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here

X₀ = init(n[1],k)         # batch di k pattern (aka i dati)
y  = init(n[end],k);      # y is what we will compare X_N against (aka l'etichetta)

𝓁(x,y)  = sum(abs2,x-y)/2   # loss (errore quadratico)
𝓁′(x,y) = x .- y;           # derivata della loss (w.r.t. output layer)

X, δ = neural_net(Ws_and_bs,X₀) # Inferenza (aka forward pass)
# X e δ hanno i valori per ogni layer (servono per fare il backward pass)

([[-0.0038920596274483023 -0.006279812952603613 … -0.017758504443525884 0.004667387128659968; 0.015942179536630473 0.012570888512030568 … 0.01371093918177978 -0.004376016958212193; … ; -0.004876053489589111 -0.0053377709119724615 … 0.006703462932380859 -0.001993766218737081; 0.004328717464026433 -0.02822612573031091 … 0.005971523297016189 0.00979093132937066], [0.0006922505541532706 0.0004375130585479821 … 0.0003690441783070967 0.000772497883546509; 0.0025324123351055075 0.0030502401659387884 … 0.002670290594542111 0.002605251236228453; 0.010599262633185908 0.010263579315659264 … 0.010482523747786928 0.010725799325388485; 0.004615310341560691 0.0047452061376692615 … 0.004942921912003799 0.004541298653022729], [0.008963047232478157 0.008958803433999367 … 0.008962178437286857 0.008962599581076211; -0.009303041646536014 -0.009298023256166563 … -0.009299818583119509 -0.009303261575996927; 0.001034564510214828 0.0010303309107956291 … 0.001032601031091371 0.0010352088322960749], [-0.00376449

Adesso possiamo calcolare il gradiente della loss usando la nostra tecnica

In [6]:
## The diagonal matrix
#M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N]) # OLD!
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i]) ∘ ℛ(ones(1,k))] for i=1:N])

## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## derivata della loss (rispetto all'output layer)
g = [ fill(𝒪(),N-1) ; [𝓁′(X[N],y)] ]      

## Finalmente, il gradiente della loss, usando il backslash
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[-2.814450734892308e-7 4.3426074348366277e-7 … 5.3993442984768135e-8 -1.2440168841931427e-7; -2.2324787182547127e-7 3.444644695462312e-7 … 4.282879075265714e-8 -9.867760752209884e-8; 6.036549800473476e-8 -9.314203016234517e-8 … -1.1580688125966708e-8 2.668219502636078e-8; -2.537333597162168e-7 3.915027214543847e-7 … 4.867679903538771e-8 -1.1215334521222997e-7]; [3.156391409253484e-5; 2.5037115826100512e-5; -6.769963414364388e-6; 2.8456069333253785e-5;;];;]
 [[-1.350483245754944e-6 -6.155963518126995e-6 -2.413542258846797e-5 -1.118519359479782e-5; 5.578578001670574e-7 2.5429062907237735e-6 9.969863771461106e-6 4.620381343698259e-6; 5.140213766793724e-7 2.3430848812544036e-6 9.186432557748002e-6 4.257312096971553e-6]; [-0.0023148954734964725; 0.0009562373491668119; 0.000881096283734965;;];;]
 [[-0.0009572917471781855 0.0009937139646240731 -0.00011036357326805235]; [-0.10683827257852258;;];;]

In [7]:
for i=1:N
    print("∇J[$i]\n")
    print("\tTipo: $(typeof(∇J[i]))\t size: $(size(∇J[i]))\n")
    print("\t\t∇J[$i][1] ha dimensione $(size(∇J[i][1])), e n_$(i) x n_$(i-1) = $(n[i+1])x$(n[i]). Tipo $(typeof(∇J[i][1]))\n")
    print("\t\t∇J[$i][2] ha dimensione $(size(∇J[i][2])), e n_$(i) \t   = $(n[i+1]).   Tipo $(typeof(∇J[i][2]))\n")
    print("\n")
end

∇J[1]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[1][1] ha dimensione (4, 5), e n_1 x n_0 = 4x5. Tipo Matrix{Float64}
		∇J[1][2] ha dimensione (4, 1), e n_1 	   = 4.   Tipo Matrix{Float64}

∇J[2]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[2][1] ha dimensione (3, 4), e n_2 x n_1 = 3x4. Tipo Matrix{Float64}
		∇J[2][2] ha dimensione (3, 1), e n_2 	   = 3.   Tipo Matrix{Float64}

∇J[3]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[3][1] ha dimensione (1, 3), e n_3 x n_2 = 1x3. Tipo Matrix{Float64}
		∇J[3][2] ha dimensione (1, 1), e n_3 	   = 1.   Tipo Matrix{Float64}



### Check

In [103]:
# flatten(J) = vcat((x->x[:]).(vcat(J...))...)

function flatten(J) 
    # create a 2N x 1 matrix of matrices [W₁; B₁; ... Wₙ]
    stackedWandB = vcat(J...)       # this is a Matrix{Matrix{Float64}}
    # flatten out the matrices inside the stack
    unstackedStack = (x->x[:]).(stackedWandB)   # this is a Matrix{Vector{Float64}}

    # stack all the vectors that are in the `unstackedStack` matrix
    return vcat(unstackedStack...)
end

flatten (generic function with 1 method)

In [104]:
print("|| ∇J || = $(norm(flatten(∇J)))\n")

|| ∇J || = 0.10688023889755821


Calcoliamo il gradiente usando il metodo delle differenze finite, per controllare il risultato

In [105]:
#∇Jfd is the gradient calculated with finite differences method

# creiamo uno zero della stessa dimensione del gradiente
∇Jfd = Ws_and_bs*0
ϵ = Ws_and_bs*0
#𝜀 = .0001
𝜀 = √eps() #* norm(flatten(∇J))
for i=1:length(Ws_and_bs), wb=1:2
    for j=1:length(ϵ[i][wb])    
        ϵ[i][wb][j] = 𝜀
        # Derivata direzionale approssimata con differenze finite
        ∇Jfd[i][wb][j] = (𝓁(neural_net(Ws_and_bs+ϵ,X₀)[1][N],y) .- 𝓁(neural_net(Ws_and_bs-ϵ,X₀)[1][N],y)) / (2𝜀)
        ϵ[i][wb][j] = .0
    end
end

In [106]:
for i=1:N
    print("∇Jfd[$i]\n")
    print("\tTipo: $(typeof(∇Jfd[i]))\t size: $(size(∇Jfd[i]))\n")
    print("\t\t∇Jfd[$i][1] ha dimensione $(size(∇Jfd[i][1])), e n_$(i) x n_$(i-1) = $(n[i+1])x$(n[i]). Tipo $(typeof(∇Jfd[i][1]))\n")
    print("\t\t∇Jfd[$i][2] ha dimensione $(size(∇Jfd[i][2]))  , e n_$(i) \t     = $(n[i+1]).   Tipo $(typeof(∇Jfd[i][2]))\n")
    print("\n")
end

∇Jfd[1]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[1][1] ha dimensione (4, 5), e n_1 x n_0 = 4x5. Tipo Matrix{Float64}
		∇Jfd[1][2] ha dimensione (4,)  , e n_1 	     = 4.   Tipo Vector{Float64}

∇Jfd[2]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[2][1] ha dimensione (3, 4), e n_2 x n_1 = 3x4. Tipo Matrix{Float64}
		∇Jfd[2][2] ha dimensione (3,)  , e n_2 	     = 3.   Tipo Vector{Float64}

∇Jfd[3]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[3][1] ha dimensione (1, 3), e n_3 x n_2 = 1x3. Tipo Matrix{Float64}
		∇Jfd[3][2] ha dimensione (1,)  , e n_3 	     = 1.   Tipo Vector{Float64}



Interessante notare un leggero mismatch di tipi tra `∇J` e `∇Jfd`, ma restano compatibili. 

Vediamo l'errore assoluto e l'errore relativo

In [108]:
print("||∇J - ∇Jfd|| = $(norm(flatten(∇J)-flatten(∇Jfd)))\n")
print("||∇J - ∇Jfd|| / ||∇J|| = $(norm(flatten(∇J)-flatten(∇Jfd)) / norm(flatten(∇J)))\n")

||∇J - ∇Jfd|| = 5.0046585346007955e-11
||∇J - ∇Jfd|| / ||∇J|| = 4.682491905166514e-10


Il fatto che l'errore sia molto piccolo suggerisce che il calcolo del gradiente di `J` tramite soluzione del sistema lineare sia corretto.

## Errori, e problemi

### In generale, moltiplicazione tra matrici di operatori

---

### Moltiplicare matrici di operatori

E' possibile moltiplicare ($\equiv$ comporre) matrici di operatori, usando le regole a nostra disposizione? E eventualmente aggiungendone di nuove?

In [22]:
Mnuova = Diagonal([ℋ(δ[i]) ∘ ℛ(X[i-1]) for i=1:N])

3×3 Diagonal{Operator, Vector{Operator}}:
 ℋ(4, 10)∘ℛ(5, 10)  ⋅                  ⋅
 ⋅                  ℋ(3, 10)∘ℛ(4, 10)  ⋅
 ⋅                  ⋅                  ℋ(1, 10)∘ℛ(3, 10)

In [23]:
try
    ImL * Mnuova
catch e
    showerror(stdout, e)
end

3×3 Bidiagonal{Operator, Vector{Operator}}:
 I∘ℋ(4, 10)∘ℛ(5, 10)                  …  ⋅
 -ℋ(3, 10)∘ℒ(3, 4)∘ℋ(4, 10)∘ℛ(5, 10)     ⋅
 ⋅                                       I∘ℋ(1, 10)∘ℛ(3, 10)

Come glielo spieghiamo che quell'elemento lì in realtà è zero??

**AGGIUSTARE**: Dando le martellate giuste, qualcosa si può fare

In [24]:
*(A::Operator, B::Operator) = A ∘ B

* (generic function with 191 methods)

In [25]:
+(A::Operator, B::Operator) = Operator(X->(A.op(X) + B.op(X)), X->(A.adj(X) + B.adj(X)), A.sym*" + "*B.sym)

+ (generic function with 193 methods)

In [26]:
Matrix(ImL) * Matrix(Mnuova)

3×3 Matrix{Operator}:
 𝒪∘𝒪 + 𝒪∘𝒪 + I∘ℋ(4, 10)∘ℛ(5, 10) + 𝒪                  …  𝒪∘ℋ(1, 10)∘ℛ(3, 10) + 𝒪∘𝒪 + I∘𝒪 + 𝒪
 𝒪∘𝒪 + I∘𝒪 + -ℋ(3, 10)∘ℒ(3, 4)∘ℋ(4, 10)∘ℛ(5, 10) + 𝒪     𝒪∘ℋ(1, 10)∘ℛ(3, 10) + I∘𝒪 + -ℋ(3, 10)∘ℒ(3, 4)∘𝒪 + 𝒪
 I∘𝒪 + -ℋ(1, 10)∘ℒ(1, 3)∘𝒪 + 𝒪∘ℋ(4, 10)∘ℛ(5, 10) + 𝒪     I∘ℋ(1, 10)∘ℛ(3, 10) + -ℋ(1, 10)∘ℒ(1, 3)∘𝒪 + 𝒪∘𝒪 + 𝒪

In [27]:
ImL * M

DimensionMismatch: DimensionMismatch: incompatible dimensions for matrix multiplication: tried to multiply a matrix of size (3, 4) with a matrix of size (1, 2). The second dimension of the first matrix: 4, does not match the first dimension of the second matrix: 1.